In [1]:
class Sentencizer:
    """"
    Class for splitting paragraphs to sentences
    """

    def __init__(self, _text, split_chars=['.', '?', '؟', '!', ':']):
        self.sentences = []
        self.text = str(_text)
        self._split_chars = split_chars
        self._sentencize()

    def _sentencize(self):

        """"
        Split sentences according to the split chars we have
        and also handle some special cases for them
        """
        fullStopIndex = findOccurrences(self.text, '.')  # get the indices of all occurrences of (.) to check
        text = self.text
        text = fullStopCheck(text, fullStopIndex)
        for character in self._split_chars:
            text = text.replace(character, character + "</>")
        text = text.replace('<D>', '.')
        self.sentences = [x.strip() for x in text.split("</>") if x != '']

class tokenization:
    """"
    CLass to create tokens from sentences
    """
    def __init__(self, text, split_tokens=[' ', '-']):

        self.tokens = []
        # Separate text into paragraphs at first
        self.paragraphs = ' '.join([prop_paragraph for prop_paragraph in text.split('\n') if len(prop_paragraph)>1])
        self.sentences = Sentencizer(self.paragraphs).sentences
        self._split_tokens = split_tokens
        self._punctuations = """'!"#$%&'()*+,«».؛،/:؟?@[\]^_`{|}~”“"""
        self._tokenize()

    def _tokenize(self):
        """"
         return tokens from the given sentences
        """
        for text in self.sentences:

            for punctuation in self._punctuations:      # Search for the punctuations inside the sentence
                text = text.replace(punctuation, " " + punctuation + " ")

            for delimiter in self._split_tokens:
                text = text.replace(delimiter, '</>')

            token = [x.strip() for x in text.split('</>') if x != '' and (x not in self._punctuations)]
            self.tokens.append(token)


def findOccurrences(s, ch):

    """"
     Finds all occurrences of a char and returns all indices
    """
    return [i for i, letter in enumerate(s) if letter == ch]


def fullStopCheck(text, indices):
    """"
    Check for all the cases were the full stop doesn't mean the end of the sentence
    example : (ق.م), (أ.د.سعيد), (د.توفيق)
    """
    for index in indices:
        if text[index - 1] == '.':  # replace multiple dots (....) with single dot (.)
            text = text[0:index] + text[index + 1:]

        elif text[index - 2] == ' ' or index - 1 == 0 or text[index - 2] == '.': # mark all special fullstops 
            text = text[0:index] + '<D>' + text[index + 1:]

    return text

In [5]:
from camel_tools.tokenizers.word import *

In [6]:
text = """قراءة د. أحمد خالد توفيق في رواية «جمهورية كأن» للروائي علاء الأسواني، المشروع الذي ظل يتحدث ولم يفارق ذهنه 
لحظة.قرأت القصة .. الحقيقة التي لا يماري فيها أحد هي أن الرجل ممتع حقًا. وهو مصر على أن يظل ممتعًا مهما كره النقاد 
ذلك. هو يؤمن بفن الرواية بتعريفه العتيق؛ أي (الحكي). أن تنتظر حتى الليل في لهفة لتعرف ما حدث . وهو مصر على الطريقة 
الديكنزية في أن يجعل الرواية متحفًا للشخصيات الفريدة الممتعة. بحيث أنك قد تنسى الرواية لكن الشخصيات تلاحقك طيلة 
الوقت.  هناك مكونات وأجزاء معينة يستعملها علاء الأسواني مثل الميكانو من قصة لأخرى، فيعيد ترتيبها ليخلق مواقف جديدة. 
مثلًا شخصية الضابط المتدين الذي يعذب الأبرياء ولا يفوت فرضًا – الثري الراقي الذي يعد بالمفهوم التقليدي فاسد الأخلاق، 
لكنه أكثر شخصيات القصة طهرًا وصدقًا، وبالطبع هناك الحب الجسدي بينه وبين خادمة أو مدبرة منزل نبيلة تحبه فعلًا، وهناك 
ضغوط المجتمع ضده بين أخت مثل دولت أو زوجة أو أولاد.. هناك علاقات تدور في قمة المجتمع فوق رءوسنا ولا نعرف عنها أي شيء . 
لم يستعمل الأسواني قطعة ميكانو الشاب الشاذ جنسيًا هذه المرة، وإن استخدم قطعة ميكانو مهمة جدًا هي ضحية الأمن الذي آمن 
بالعدالة ثم أدرك مع الوقت أن ديانة الدولة هي الظلم، وأن الدم هو الحل الوحيد. لن أعطي تفاصيل حتى لا أدمر نهاية 
الرواية، لكنها نهاية قوية جدًا."""

In [7]:
camels = simple_word_tokenize(text)

# Using Omdena tokenizer
tokenizer = tokenization(text)
omdenas = tokenizer.tokens
omdenas_falttened = [x for sublist in omdenas for x in sublist]

In [14]:
import pyarabic.araby as araby
pyarab = araby.tokenize(text)

In [9]:
from rouge import Rouge
rouge_ = Rouge()
score = rouge_.get_scores(' '.join(omdenas_falttened), ' '.join(camels))
rouge1 = score[0].get('rouge-1')
print(rouge1)

{'f': 0.9409090859128101, 'p': 0.9672897196261683, 'r': 0.915929203539823}


In [16]:
from rouge import Rouge
rouge_ = Rouge()
score = rouge_.get_scores(' '.join(omdenas_falttened), ' '.join(pyarab))
rouge1 = score[0].get('rouge-1')
print(rouge1)

{'f': 0.9452054744546612, 'p': 0.9672897196261683, 'r': 0.9241071428571429}


In [11]:
camels

['قراءة',
 'د',
 '.',
 'أحمد',
 'خالد',
 'توفيق',
 'في',
 'رواية',
 '«',
 'جمهورية',
 'كأن',
 '»',
 'للروائي',
 'علاء',
 'الأسواني',
 '،',
 'المشروع',
 'الذي',
 'ظل',
 'يتحدث',
 'ولم',
 'يفارق',
 'ذهنه',
 'لحظة',
 '.',
 'قرأت',
 'القصة',
 '.',
 '.',
 'الحقيقة',
 'التي',
 'لا',
 'يماري',
 'فيها',
 'أحد',
 'هي',
 'أن',
 'الرجل',
 'ممتع',
 'حقًا',
 '.',
 'وهو',
 'مصر',
 'على',
 'أن',
 'يظل',
 'ممتعًا',
 'مهما',
 'كره',
 'النقاد',
 'ذلك',
 '.',
 'هو',
 'يؤمن',
 'بفن',
 'الرواية',
 'بتعريفه',
 'العتيق',
 '؛',
 'أي',
 '(',
 'الحكي',
 ')',
 '.',
 'أن',
 'تنتظر',
 'حتى',
 'الليل',
 'في',
 'لهفة',
 'لتعرف',
 'ما',
 'حدث',
 '.',
 'وهو',
 'مصر',
 'على',
 'الطريقة',
 'الديكنزية',
 'في',
 'أن',
 'يجعل',
 'الرواية',
 'متحفًا',
 'للشخصيات',
 'الفريدة',
 'الممتعة',
 '.',
 'بحيث',
 'أنك',
 'قد',
 'تنسى',
 'الرواية',
 'لكن',
 'الشخصيات',
 'تلاحقك',
 'طيلة',
 'الوقت',
 '.',
 'هناك',
 'مكونات',
 'وأجزاء',
 'معينة',
 'يستعملها',
 'علاء',
 'الأسواني',
 'مثل',
 'الميكانو',
 'من',
 'قصة',
 'لأخرى',
 '،',
 'في

In [12]:
omdenas_falttened

['قراءة',
 'د',
 'أحمد',
 'خالد',
 'توفيق',
 'في',
 'رواية',
 'جمهورية',
 'كأن',
 'للروائي',
 'علاء',
 'الأسواني',
 'المشروع',
 'الذي',
 'ظل',
 'يتحدث',
 'ولم',
 'يفارق',
 'ذهنه',
 'لحظة',
 'قرأت',
 'القصة',
 'الحقيقة',
 'التي',
 'لا',
 'يماري',
 'فيها',
 'أحد',
 'هي',
 'أن',
 'الرجل',
 'ممتع',
 'حقًا',
 'وهو',
 'مصر',
 'على',
 'أن',
 'يظل',
 'ممتعًا',
 'مهما',
 'كره',
 'النقاد',
 'ذ',
 'ك',
 'هو',
 'يؤمن',
 'بفن',
 'الرواية',
 'بتعريفه',
 'العتيق',
 'أي',
 'الحكي',
 'أن',
 'تنتظر',
 'حتى',
 'الليل',
 'في',
 'لهفة',
 'لتعرف',
 'ما',
 'حدث',
 'وهو',
 'مصر',
 'على',
 'الطريقة',
 'الديكنزية',
 'في',
 'أن',
 'يجعل',
 'الرواية',
 'متحفًا',
 'للشخصيات',
 'الفريدة',
 'الممتعة',
 'بحيث',
 'أنك',
 'قد',
 'تنسى',
 'الرواية',
 'لكن',
 'الشخصيات',
 'تلاحقك',
 'طيلة',
 'ا',
 'وقت',
 'هناك',
 'مكونات',
 'وأجزاء',
 'معينة',
 'يستعملها',
 'علاء',
 'الأسواني',
 'مثل',
 'الميكانو',
 'من',
 'قصة',
 'لأخرى',
 'فيعيد',
 'ترتيبها',
 'ليخلق',
 'مواقف',
 'جديدة',
 'مثلًا',
 'شخصية',
 'الضابط',
 'المتدين',
 'ا

In [17]:
pyarab

['قراءة',
 'د',
 '.',
 'أحمد',
 'خالد',
 'توفيق',
 'في',
 'رواية',
 '«',
 'جمهورية',
 'كأن',
 '»',
 'للروائي',
 'علاء',
 'الأسواني',
 '،',
 'المشروع',
 'الذي',
 'ظل',
 'يتحدث',
 'ولم',
 'يفارق',
 'ذهنه',
 '\n',
 'لحظة',
 '.',
 'قرأت',
 'القصة',
 '..',
 'الحقيقة',
 'التي',
 'لا',
 'يماري',
 'فيها',
 'أحد',
 'هي',
 'أن',
 'الرجل',
 'ممتع',
 'حقًا',
 '.',
 'وهو',
 'مصر',
 'على',
 'أن',
 'يظل',
 'ممتعًا',
 'مهما',
 'كره',
 'النقاد',
 '\n',
 'ذلك',
 '.',
 'هو',
 'يؤمن',
 'بفن',
 'الرواية',
 'بتعريفه',
 'العتيق',
 '؛',
 'أي',
 '(',
 'الحكي',
 ').',
 'أن',
 'تنتظر',
 'حتى',
 'الليل',
 'في',
 'لهفة',
 'لتعرف',
 'ما',
 'حدث',
 '.',
 'وهو',
 'مصر',
 'على',
 'الطريقة',
 '\n',
 'الديكنزية',
 'في',
 'أن',
 'يجعل',
 'الرواية',
 'متحفًا',
 'للشخصيات',
 'الفريدة',
 'الممتعة',
 '.',
 'بحيث',
 'أنك',
 'قد',
 'تنسى',
 'الرواية',
 'لكن',
 'الشخصيات',
 'تلاحقك',
 'طيلة',
 '\n',
 'الوقت',
 '.',
 'هناك',
 'مكونات',
 'وأجزاء',
 'معينة',
 'يستعملها',
 'علاء',
 'الأسواني',
 'مثل',
 'الميكانو',
 'من',
 'قصة',
 '